* > # EE226 - Coding 2
## Streaming algorithm & Locality Sensitive Hashing

### Streaming: DGIM

DGIM is an efficient algorithm in processing large streams. When it's infeasible to store the flowing binary stream, DGIM can estimate the number of 1-bits in the window. In this coding, you're given the *stream_data.txt* (binary stream), and you need to implement the DGIM algorithm to count the number of 1-bits. Write code and ask the problems below.

### Your task

1. Set the window size to 1000, and count the number of 1-bits in the current window.

In [1]:
!pip install memory_profiler

In [2]:
import time
%load_ext memory_profiler
filename="../input/coding2/stream_data.txt"

windowsize=1000
timestamp=0
buckets_list=[]

# ws=windowsize,ts=timestamp,bl=buckets_list[]
def update_buckets_list(ws,ts,bl):
    if len(bl)>0 and ts-ws==bl[0]["timestamp"]: # drop the oldest bucket
        del bl[0]
    for i in range(len(bl)-1,1,-1):
        if bl[i]["bucket_size"]==bl[i-2]["bucket_size"]: # merge two buckets
            bl[i-2]["bucket_size"]=2*bl[i-2]["bucket_size"]
            bl[i-2]["timestamp"]=bl[i-1]["timestamp"]
            del bl[i-1]

# counts of 1-bit in this window
def count_dgim(filename):
    windowsize=1000
    timestamp=0
    buckets_list=[]
    with open(filename, 'r') as f:
        while True:
            bit=f.read(1)
            if not bit:
                break
            if bit=="1":
                bucket={"bucket_size":1,"timestamp":timestamp}
                buckets_list.append(bucket)
                timestamp=timestamp+1
                update_buckets_list(windowsize,timestamp,buckets_list)
            elif bit=="0":
                timestamp=timestamp+1
                update_buckets_list(windowsize,timestamp,buckets_list)
    for i in range(len(buckets_list)):
        sum_of_1bit_1=list(buckets_list[i]["bucket_size"] for i in range(len(buckets_list)))
        sum_of_1bit=sum(sum_of_1bit_1)-buckets_list[0]["bucket_size"]/2
    return sum_of_1bit

sum_of_1bit=count_dgim(filename)

print("The number of 1-bits in the current window using DGIM:",sum_of_1bit)
%time count_dgim(filename)
%memit count_dgim(filename)

The number of 1-bits in the current window using DGIM: 508.0
CPU times: user 196 ms, sys: 0 ns, total: 196 ms
Wall time: 196 ms
peak memory: 172.88 MiB, increment: 0.18 MiB


2. Write a function that accurately counts the number of 1-bits in the current window, and compare the difference between its running time and space and the DGIM algorithm.

In [3]:
# the accurate count of 1-bits in this window
def count_accurate(filename):
    accurate_1bits=[]
    sum_of_1bit_acc=0
    sum_of_bit=0
    windowsize=1000
    timestamp1=1
    with open(filename, 'r') as f:
        while True:
            bit=f.read(1)
            if not bit:
                break
            else:
                if bit=="1":
                    accurate_1bits.append(1)
                elif bit=="0":
                    accurate_1bits.append(0)
                if len(accurate_1bits)>1000:
                    del accurate_1bits[0]
    sum_of_1bit_acc=sum(accurate_1bits)
    return sum_of_1bit_acc

sum_of_1bit_acc=count_accurate(filename)
print("The accurate number of 1-bits in the current window:",sum_of_1bit_acc)
%time count_accurate(filename)
%memit count_accurate(filename)
# It uses less time than DGIM with about the same space.

The accurate number of 1-bits in the current window: 391
CPU times: user 41.1 ms, sys: 0 ns, total: 41.1 ms
Wall time: 41.7 ms
peak memory: 172.94 MiB, increment: 0.01 MiB


It uses less time than DGIM with about the same space.

### Locality Sensitive Hashing

The locality sensitive hashing (LSH) algorithm is efficient in near-duplicate document detection. In this coding, you're given the *docs_for_lsh.csv*, where the documents are processed into set of k-shingles (k = 8, 9, 10). *docs_for_lsh.csv* contains 201 columns, where column 'doc_id' represents the unique id of each document, and from column '0' to column '199', each column represents a unique shingle. If a document contains a shingle ordered with **i**, then the corresponding row will have value 1 in column **'i'**, otherwise it's 0. You need to implement the LSH algorithm and ask the problems below.

### Your task

Use minhash algoirthm to create signature of each document, and find 'the most similar' documents under Jaccard similarity. 
Parameters you need to determine:
1) Length of signature (number of distinct minhash functions) *n*. Recommanded value: n > 20.

2) Number of bands that divide the signature matrix *b*. Recommanded value: b > n // 10.

In [4]:
import pandas as pd
import random
import numpy as np
from itertools import chain
import hashlib
from sklearn.metrics import jaccard_score
import scipy.spatial.distance as dist

data=pd.read_csv('../input/coding2/docs_for_lsh.csv',index_col=0)
print(data.head(5))

        0  1  2  3  4  5  6  7  8  9  ...  190  191  192  193  194  195  196  \
doc_id                                ...                                      
0       1  0  1  0  0  0  1  1  0  1  ...    0    0    0    0    0    0    0   
1       1  1  0  1  0  1  1  0  0  0  ...    0    0    0    0    0    0    0   
2       1  0  1  0  1  1  0  1  0  0  ...    0    0    0    0    0    0    0   
3       0  0  0  1  1  0  1  0  1  1  ...    0    0    0    0    0    0    0   
4       0  0  1  1  1  1  0  1  0  0  ...    0    0    0    0    0    0    0   

        197  198  199  
doc_id                 
0         0    0    0  
1         0    0    0  
2         0    0    0  
3         0    0    0  
4         0    0    0  

[5 rows x 200 columns]


In [5]:
data = pd.read_csv("../input/coding2/docs_for_lsh.csv")
data_list=[]
for i in range(200):
    data_list.append(data[str(i)].tolist())
length=len(data_list[0])
n_perms =40 # Length of signature
n_samples=200 # Count of docs
bands=500000 # Number of bands
hash_functions=[] 

# generate random lists as hash functions
def random_list(start,stop,length):
    if length>=0:
        length=int(length)
    start, stop = (int(start), int(stop)) if start <= stop else (int(stop), int(start))
    random_list=random.sample(range(start,stop), length)
    return random_list

for i in range(n_perms):
    hash_functions.append(random_list(0,length,length))

In [6]:
#generate a signature matrix
signature_matrix=[[] for i in range(n_perms)]
for l in range(n_perms):
    for i in range(n_samples):
        min_index=length
        for j in range(length):
            if data_list[i][j]==1:
                k=j
                if hash_functions[l][k] < min_index:
                    min_index=hash_functions[l][k]
        signature_matrix[l].append(min_index)

signature_matrix=np.array(signature_matrix)
print(signature_matrix[0])

[  3  43   9   3  57   9   9   3   3  34   3   9   9   3   9   3   3   3
   3   9   2   2   2  24   2   2   2  46  26  24   2  52   2  24   2  46
  24   2  24  24   0  27  19   0   0   4   0   0   4   6  12   0   0   4
   0   0   4   0   0   0   1   1  40   1  91   1   1   1  40   1  28  28
  28   1  28   1  28  28  28   1  37  18  31  18  18  31  18  31  18  31
  18  37  18  44  18  45  31  18  31  18  20  16  16  16  16  17  16  60
  16  17  16  16  17  20  23  20  16  17  17  16  78  11  11   5   5  11
   5   5  11   5   5  76   5  66   5  11   5   5   5  65  54  54  15  15
  15  15  95  93  15  15  15  54  15  15 121  75  15  90  15  15   7   7
   7   7  13  13   7   7   7   8   7   8   7   8   7   8   8   7  13   7
  30  35  35  35  30  30  30  30  59  30  59  59  42  35  30  30  30  88
  30  30]


In [7]:
# minHash: input a signature matrix,bands and rows of each band
def minHash(s, b, r):
    hash_buckets={} # a dictionary, buckets as key, docs as value
    start,end=0,r
    cnt=0
    while end<=s.shape[0]:
        cnt=cnt+1
        for i in range(s.shape[1]):
            hashObj=hashlib.md5()  # inserted hash
            band=str(s[start:start+r,i])+str(cnt) # distinguish each band
            hashObj.update(band.encode())
            bucket=hashObj.hexdigest() # return the bucket
            if bucket not in hash_buckets: # add a new bucket
                hash_buckets[bucket]=[i]
            elif i not in hash_buckets[bucket]: # add element to an existing bucket
                hash_buckets[bucket].append(i)
        start=start+r
        end=end+r
    return hash_buckets

In [8]:
# get the hash_buckets, 500000 bands
minhash=minHash(signature_matrix,500000,2) 
sig_matrix_T=signature_matrix.T

# show the top 10 hash_buckets
for i, (k, v) in enumerate(minhash.items()):
    if i in range(0, 10):
        print(k, v)

# calculate the similarity of each two docs in one bucket
similar_matrix=np.array(np.zeros([n_samples,n_samples]))
for value in minhash.values():
    temp=value
    if len(temp)>1:
        for i in range(0, len(temp)):
            for j in range(i + 1, len(temp)): # for each two docs in one bucket
                matv = np.array([sig_matrix_T[temp[i]],sig_matrix_T[temp[j]]])
                ds=dist.pdist(matv,'jaccard') # Jaccard similarity
                similar_matrix[temp[i]][temp[j]]=ds
print('')
print("The most similar score:",similar_matrix.max()) # the most similar pairs
similar_pairs_loc=np.where(similar_matrix==similar_matrix.max())
pairs=[]
for i in range(len(similar_pairs_loc[0])):
    pairs.append((similar_pairs_loc[0][i],similar_pairs_loc[1][i]))
print("The most similar pairs:",pairs)

18c92a4b95979e57f0b6fb4ce1ec872f [0]
81208555d35bc0c40862976cd826ebf4 [1]
8ba21865ea0aa5c0f7afb47b9751b485 [2, 12]
36776040878af6e30c12244b01db980f [3, 7]
dd7e6826a92573f894ca8c0189b68cba [4]
b5b79b629fc44cdb08985fdc2a5fea24 [5, 11, 14]
a80e63aad3b0b42bab76f2541da96306 [6]
2996c6efe596ba957b2b027e3a2f45f1 [8, 13, 15, 16, 17, 18]
3c8c7e575fd2f54867b820480279631a [9]
a967cb42d47494912d43b6efd964443e [10]

The most similar score: 0.868421052631579
The most similar pairs: [(3, 11)]


Problem: For document 0 (the one with id '0'), list the **30** most similar document ids (except document 0 itself). You can valid your results with the [sklearn.metrics.jaccard_score()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.jaccard_score.html) function.

Tips: You can adjust your parameters to hash the documents with similarity *s > 0.8* into the same bucket.

In [9]:
similar_pairs_for_zero=list(similar_matrix[0])
sort_for_zero=np.argsort(similar_pairs_for_zero)[::-1]
result=sort_for_zero[0:30]
print("The 30 most similar document ids to document 0:")
for i in range(30):
    print("Document",result[i]," with similarity score",similar_matrix[0][result[i]])

The 30 most similar document ids to document 0:
Document 17  with similarity score 0.7894736842105263
Document 2  with similarity score 0.7894736842105263
Document 12  with similarity score 0.7837837837837838
Document 19  with similarity score 0.7631578947368421
Document 16  with similarity score 0.7567567567567568
Document 1  with similarity score 0.725
Document 3  with similarity score 0.717948717948718
Document 6  with similarity score 0.7105263157894737
Document 18  with similarity score 0.6842105263157895
Document 15  with similarity score 0.6756756756756757
Document 4  with similarity score 0.675
Document 5  with similarity score 0.6666666666666666
Document 10  with similarity score 0.6578947368421053
Document 13  with similarity score 0.6486486486486487
Document 8  with similarity score 0.6410256410256411
Document 11  with similarity score 0.6216216216216216
Document 14  with similarity score 0.5384615384615384
Document 68  with similarity score 0.0
Document 75  with similarity 